In [12]:
import pandas as pd


# 读取 Excel 文件
file1_path = '../附件1.xlsx'
file2_path = '../附件2.xlsx'

# 读取所有sheet
land_data = pd.read_excel(file1_path, sheet_name='乡村的现有耕地')  # 附件1中包含地块信息
land_crop_data = pd.read_excel(file1_path, sheet_name='乡村种植的农作物')  # 附件1中种植信息

crop_planting_data = pd.read_excel(file2_path, sheet_name='2023年的农作物种植情况')  # 附件2中2023年作物种植情况
crop_stats_data = pd.read_excel(file2_path, sheet_name='2023年统计的相关数据')  # 附件2中2023年的相关数据


# 定义地块前缀对应的地块类型映射
land_type_mapping = {
    'A': '平旱地',
    'B': '梯田',
    'C': '山坡地',
    'D': '水浇地',
    'E': '普通大棚 ',
    'F': '智慧大棚'
    
}

# 从地块名称中提取前缀并映射到地块类型
land_data = land_data.drop('说明 ', axis=1)
crop_stats_data = crop_stats_data.drop(['序号', '作物编号'], axis=1)
land_data['地块前缀'] = land_data['地块名称'].str[0]  # 提取第一个字符作为前缀
land_data['地块类型'] = land_data['地块前缀'].map(land_type_mapping)  # 映射到地块类型

# 合并地块信息和种植信息（基于地块类型）

merged_data = land_data.merge(crop_stats_data, on='地块类型', how='left')

# 根据种植季次生成季节限制
def generate_season_restriction(row):
    # 如果种植季次是"单季"，季节限制设为1
    if row['种植季次'] == '单季':
        row['季节限制'] = 1
        row['第一季单价(元)'] = row['销售单价/(元/斤)']
        row['第二季单价(元)'] = 0
    # 如果种植季次是"第一季"或"第二季"，季节限制设为2（表示可种两季）
    elif row['种植季次'] in ['第一季', '第二季']:
        row['季节限制'] = 2
        if row['种植季次'] == '第一季':
            row['第一季单价(元)'] = row['销售单价/(元/斤)']
        else:
            row['第二季单价(元)'] = row['销售单价/(元/斤)']
            
    return row

# 应用生成季节限制的逻辑
merged_data = merged_data.apply(generate_season_restriction, axis=1)
merged_data =merged_data.fillna(0)
df=merged_data 

#处理价格区间
df['第一季最小单价(元)']=df['第一季单价(元)'].str.split('-', expand=True)[0].astype(float, errors='ignore')
df['第一季最大单价(元)']=df['第一季单价(元)'].str.split('-', expand=True)[1].astype(float, errors='ignore')
df['第二季最小单价(元)']=df['第二季单价(元)'].str.split('-', expand=True)[0].astype(float, errors='ignore')
df['第二季最大单价(元)']=df['第二季单价(元)'].str.split('-', expand=True)[1].astype(float, errors='ignore')
df=df.fillna(0)

#2023年产量
crop_recom=pd.read_excel("../crop_commd.xlsx")

In [13]:
import pulp as lp

# 将DataFrame转化为优化模型需要的字典格式
data = {}
for idx, row in df.iterrows():
    data[(row['地块名称'], row['作物名称'])] = (
        row['地块类型'],
        row['亩产量/斤'], 
        row['种植成本/(元/亩)'],
        (row['第一季最小单价(元)']+row['第一季最大单价(元)'])/2,
        (row['第二季最小单价(元)']+row['第二季最大单价(元)'])/2,
        row['季节限制']
    )

# 地块和作物名称
land_plots = df['地块名称'].unique().tolist()
crops = df['作物名称'].unique().tolist()
land_area = df.set_index('地块名称')['地块面积/亩'].to_dict()
# 定义豆类作物的列表
legumes = ['黄豆', '黑豆','红豆','绿豆','爬豆','豇豆','刀豆','芸豆',]  # 豆类作物
vegetables = ['豇豆','刀豆','芸豆','土豆','西红柿',
              '茄子','菠菜','青椒','菜花','包菜',
              '油麦菜','小青菜','黄瓜','生菜','辣椒',
              '空心菜','黄心菜','芹菜','大白菜','白萝卜','红萝卜'
]  # 蔬菜类作物列表
fungi = ['香菇','榆黄菇','白灵菇','羊肚菌']  # 食用菌类作物列表

years = [2024, 2025, 2026, 2027, 2028, 2029, 2030]  # 年份


# 假设一些市场需求
crop_demand = {}
for idx, row in crop_recom.iterrows():
    crop_demand[row['作物名称']] = row['总产量'] 

crop_demand = {k.strip(): v for k, v in crop_demand.items()}
crops= [item.strip() for item in crops]

In [14]:
# 辅助函数：判断是否为有效的地块和作物的季节种植组合
def is_valid_plot(i, j, s):
    # 检查地块类型和作物季节限制
    plot_type = data[(i, j)][0]  # 地块类型
    if plot_type in ['平旱地', '梯田', '山坡地'] and s == 2:
        return False  # 平旱地、梯田和山坡地只能种一季
    if plot_type == '水浇地' and s == 1 and j == '蔬菜':
        return False  # 水浇地第一季不能种蔬菜
    if plot_type == '普通大棚' and j == '水稻':
        return False  # 普通大棚不能种水稻
    return True


In [16]:
#决策函数
def decision():
    import pulp as lp
    import numpy as np

    # 定义模拟的次数
    num_simulations = 10

    # 存储每次模拟的总利润
    profits = {}
    decision={}
    detailed_info={}

    for sim in range(num_simulations):
        print(f"Simulation {sim + 1}/{num_simulations}")

        # 1. 模拟不确定性参数的波动
        # 销售量的变化
        future_crop_demand = {}
        for j in crops:
            if j in ['小麦', '玉米']:  # 对小麦和玉米，年增长5%~10%
                growth_rate = np.random.uniform(0.05, 0.10)
                for y in years:
                    future_crop_demand[j, y] = crop_demand[j] * (1 + growth_rate)**(y - 2023)
            else:  # 对其他作物 ±5%的变化
                for y in years:
                    
                    future_crop_demand[j, y] = crop_demand[j] * np.random.uniform(0.95, 1.05)

        # 亩产量的变化，每年±10%的变化
        future_yield = {}
        for i, j in data:
            for y in years:
                future_yield[i, j, y] = data[(i, j)][1] * np.random.uniform(0.9, 1.1)

        # 种植成本年增长5%
        future_costs = {}
        for i, j in data:
            for y in years:
                future_costs[i, j, y] = data[(i, j)][2] * (1.05)**(y - 2023)

        # 销售价格变化
        future_prices = {}
        for i, j in data:
            for y in years:
                if j in ['小麦', '玉米']:  # 粮食价格基本稳定
                    future_prices[i, j, 1, y] = data[(i, j)][3]
                    future_prices[i, j, 2, y] = data[(i, j)][4]
                elif j in vegetables:  # 蔬菜类每年增长5%
                    future_prices[i, j, 1, y] = data[(i, j)][3] * (1.05)**(y - 2023)
                    future_prices[i, j, 2, y] = data[(i, j)][4] * (1.05)**(y - 2023)
                elif j in fungi:  # 食用菌价格每年下降1%-5%
                    decline_rate = np.random.uniform(0.01, 0.05)
                    future_prices[i, j, 1, y] = data[(i, j)][3] * (1 - decline_rate)**(y - 2023)
                    future_prices[i, j, 2, y] = data[(i, j)][4] * (1 - decline_rate)**(y - 2023)
                else:  # 默认保持不变
                    future_prices[i, j, 1, y] = data[(i, j)][3]
                    future_prices[i, j, 2, y] = data[(i, j)][4]

        # 2. 创建新的问题
        prob = lp.LpProblem("Crop_Planting_Optimization", lp.LpMaximize)
        min_area_thresholdd=0
        # 决策变量：x[i][j][s][y] 表示在第 i 块地上种植第 j 种作物的面积，s 表示季节 (1 或 2)，y 表示年份
        x = lp.LpVariable.dicts(
            "x", 
            ((i, j, s, y) for i in land_plots for j in crops for s in [1, 2] for y in years
            if (i, j) in data and (s == 1 or data[(i, j)][5] == 2) and data[(i, j)][3 if s == 1 else 4] != 0), 
            lowBound=0, cat="Continuous")
        #第一阶段目标优化
        # 确保每块地在3年内的所有面积都至少种植一次豆类
        for i in land_plots:
            for y in range(2023, max(years) - 4):  # 遍历每个连续的三年窗口
                prob += lp.lpSum(
                    x[(i, j, s, y+k)] for j in legumes for s in [1, 2] for k in range(3)
                    if (i, j, s, y+k) in x and (s == 1 or data[(i, j)][5] == 2)  # 只遍历允许的季节
                ) >= land_area[i], f"Legume_Rotation_{i}_{y}"  # 设置为总面积
        
        for i in land_plots:
            for y in years:  # 遍历每个年份
                prob += lp.lpSum(x[(i, j, s, y)] for j in crops for s in [1, 2] 
                                if (i, j) in data and (s == 1 or data[(i, j)][5] == 2) 
                                and (i, j, s, y) in x) <= land_area[i], f"Land_Area_Constraint_{i}_{y}"
        
        # 定义 waste 变量，去掉作物名称的空格
        waste = lp.LpVariable.dicts("waste", ((j.strip(), y) for j in crops for y in years), lowBound=0, cat="Continuous")
        # 去除作物名称中的空格
        for j in crops:
            j_clean = j.strip()  # 去掉作物名称前后的空格
            for y in years:
                for i in land_plots:
                    for s in [1, 2]:  # 添加季节 s 来确保约束名称唯一
                        # 添加条件到 for 循环中，确保只处理符合条件的组合
                        prob += lp.lpSum(
                            x.get((i, j_clean, s, y), 0) * data.get((i, j_clean), [0, 0])[1]
                            for i in land_plots if (i, j_clean) in data and (s == 1 or data[(i, j_clean)][5] == 2)
                        ) <= crop_demand.get(j_clean, 0) + waste.get((j_clean, y), 0), \
                        f"Crop_Yield_Limit_With_Waste_{i}_{j_clean}_{y}_S{s}"  # 加上季节 s，确保唯一
        
        # 目标函数：每年收益最大化
        prob += lp.lpSum(
            (x.get((i, j, s, y), 0) * future_yield.get((i, j, y), 0) * future_prices.get((i, j, s, y), 0)  # 销售收益
            - future_costs.get((i, j, y), 0) * x.get((i, j, s, y), 0)  # 减去种植成本
            - waste.get((j, y), 0) * 0.5 * future_prices.get((i, j, s, y), 0))  # 超过部分按50%价格出售
            for i in land_plots for j in crops for s in [1, 2] for y in years
            if (i, j) in data and (s == 1 or data[(i, j)][5] == 2)  # 只允许合法种植
            and is_valid_plot(i, j, s)
        ), "Maximize_Total_Profit_With_Discounted_Sales"

       
        # 约束1：每块地的总种植面积不能超过可用面积
        # 修改 Land_Area_Constraint 名称，添加季节 s
        for i in land_plots:
            for y in years:  # 遍历每个年份
                for s in [1, 2]:  # 添加季节
                    prob += lp.lpSum(x[(i, j, s, y)] for j in crops  # 获取种植面积
                                    if (i, j) in data and (s == 1 or data[(i, j)][5] == 2) 
                                    and (i, j, s, y) in x) <= land_area[i], \
                                    f"Land_Area_Constraint_{i}_{y}_S{s}"
        
        #约束条件2：种植不能太分散
        for j in crops:
            for i in land_plots:
                for y in years:
                    # 根据地块面积动态调整最小种植面积
                    if land_area[i] > 10:  # 面积大于10亩的地块
                        min_area_threshold = 0.1 * land_area[i]  # 地块总面积的10%
                    else:  # 面积较小的地块
                        min_area_threshold = 0.05 * land_area[i]  # 地块总面积的5%
                    
                    # 确保每个约束的名称唯一，添加季节(s)变量到约束名中
                    for s in [1, 2]:
                        prob += lp.lpSum(x[(i, j, s, y)] for s in [1, 2] if (i, j, s, y) in x) >= min_area_thresholdd, \
                                f"Min_Concentrated_Area_Constraint_{i}_{j}_{y}_S{s}"
        
        #约束3：禁止同一年内同一地块种植同一作物在两季
        for i in land_plots:
            for j in crops:
                for y in years:  # 遍历每一年
                    if (i, j, 1, y) in x and (i, j, 2, y) in x:  # 检查变量是否存在
                        prob += x[(i, j, 1, y)] + x[(i, j, 2, y)] <= land_area[i], f"No_Same_Crop_Two_Seasons_{i}_{j}_Year_{y}"

        #约束4：禁止连续两年种植同一种作物（防止重茬)
        for i in land_plots:
            for j in crops:
                for y in years[:-1]:  # 遍历年份，检查连续两年的种植情况
                    # 第一季重茬限制
                    if (i, j, 1, y) in x and (i, j, 1, y+1) in x:
                        prob += x[(i, j, 1, y)] + x[(i, j, 1, y+1)] <= land_area[i], f"No_Continuous_Planting_{i}_{j}_{y}_S1"
                    # 第二季重茬限制
                    if (i, j, 2, y) in x and (i, j, 2, y+1) in x:
                        prob += x[(i, j, 2, y)] + x[(i, j, 2, y+1)] <= land_area[i], f"No_Continuous_Planting_{i}_{j}_{y}_S2"


        # 3. 求解问题
        prob.solve()

        # 输出求解状态
        print(f"状态: {lp.LpStatus[prob.status]}")
        yearly_profit = [] 
        decision_yearly=[]
        
        # 检查求解状态，并保存模拟的利润和决策
        if lp.LpStatus[prob.status] == 'Optimal':
            yearly_profit = []
            decision_yearly = []
            detailed_yearly = []  # 存储详细信息

            for y in years:
                year_profit = 0
                year_detailed = []

                for i in land_plots:
                    for j in crops:
                        for s in [1, 2]:
                            if (i, j, s, y) in x:
                                planted_area = x[(i, j, s, y)].varValue
                                if planted_area > 0:
                                    # 计算利润
                                    year_profit += planted_area * future_yield[i, j, y] * future_prices[i, j, s, y]
                                    # 记录种植决策
                                    decision_yearly.append((i, j, s, y, planted_area))
                                    # 记录详细信息：成本、销售量、单价、亩产量、收益等
                                    cost = future_costs[i, j, y]
                                    #revenue = planted_area * future_yield[i, j, y] * future_prices[i, j, s, y]
                                    demand = future_crop_demand[j, y]
                                    yield_per_acre = future_yield[i, j, y]  # 记录亩产量
                                    
                                    year_detailed.append({
                                        '地块': i, 
                                        '作物': j, 
                                        '季节': s, 
                                        '年份': y,
                                        '种植面积': round(planted_area, 1),  # 保留一位小数
                                        '亩成本': round(cost, 1),  # 保留一位小数
                                        '亩产量': round(yield_per_acre, 1),  # 保留一位小数
                                        '销售价格': round(future_prices[i, j, s, y], 2),  
                                        '销售量': round(demand, 1)  # 保留一位小数
                                    })
                
                yearly_profit.append(year_profit)
                detailed_yearly.append(year_detailed)
            
            # 保存每次模拟的利润和决策
            profits[f"{sim + 1}"] = yearly_profit
            decision[f"{sim + 1}"] = decision_yearly
            detailed_info[f"{sim + 1}"] = detailed_yearly
        else:
            print(f"Simulation {sim + 1} did not find an optimal solution.")
        
    # 找出最小利润对应的模拟
    worst_simulation = min(profits, key=lambda k: sum(profits[k]))

    # 输出最差情况的种植策略和详细信息
    #print(f"最差利润对应的模拟: {worst_simulation}")
    #print(f"最差利润: {sum(profits[worst_simulation])}")
    #print(f"最差情况下的种植策略: {decision[worst_simulation]}")
    #print(f"最差情况下的详细信息: {detailed_info[worst_simulation]}")

    return detailed_info[worst_simulation],sum(profits[worst_simulation])

In [17]:
decision_info,poroti=decision()

Simulation 1/10
状态: Optimal
Simulation 2/10
状态: Optimal
Simulation 3/10
状态: Optimal
Simulation 4/10
状态: Optimal
Simulation 5/10
状态: Optimal
Simulation 6/10
状态: Optimal
Simulation 7/10
状态: Optimal
Simulation 8/10
状态: Optimal
Simulation 9/10
状态: Optimal
Simulation 10/10
状态: Optimal


In [9]:
#将结果输出到csv
# 展开嵌套的列表字典结构
import pandas as pd
data=decision_info
flattened_data = []
for simulation in data:
    for record in simulation:
        flattened_data.append({
            '年份': record['年份'],
            '地块名': record['地块'],
            '季节': record['季节'],
            '作物名': record['作物'],
            '种植面积': record['种植面积']
        })

# 将数据转换为 DataFrame
df = pd.DataFrame(flattened_data)
#输出为csv文件
df.to_csv("../re222.csv")